# Brian Lee - Twitter Spam Detection Through CNN Text Classifier

##Reference: [A deep learning model for Twitter spam detection](https://www.sciencedirect.com/science/article/pii/S2468696420300203)

In [ ]:
# PYTORCH
import torch
import torch.nn as nn
from torch.optim import Adam

# Other ML Tools
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from nltk.tokenize import TweetTokenizer, word_tokenize

# Basic Libraries
from collections import defaultdict
import re
import random


In [ ]:
import nltk
nltk.download('punkt')